In [6]:
from PIL import Image, ImageFont, ImageDraw 
import glob
import string
import random
import os
import shutil
import math
import tensorflow as tf
print(tf.__version__)
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, Rescaling
from tensorflow.keras import Input 
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

width = 128
height = 128

2.9.0


In [7]:
data_path = "C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data"
ext = ["jpg","png", "jpeg"];

def read_img(path):
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (width, height))
    return img

In [8]:
imgs = []
labels = []

for path in glob.glob(f'{data_path + "/True"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            print(file)
            imgs.append(read_img(file))
            labels.append(1.0)

for path in glob.glob(f'{data_path + "/False"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            imgs.append(read_img(file))
            labels.append(0.0)
            
print("Imgs len: ", len(imgs))
print("Labels len: ", len(labels))

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_01.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_02.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_03.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_04.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_05.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_06.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_07.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_08.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_32_two\image_09.png
C:/Users/Scrip0/Des

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_17.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_18.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_19.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_20.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_21.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_22.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_23.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_24.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_44_six\image_25.png
C:/Users/Scrip0/Des

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_01.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_02.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_03.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_04.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_05.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_06.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_07.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_08.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_32_56_ten\image_09.png
C:/Users/Scrip0/Des

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_25.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_26.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_27.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_28.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_29.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_04_king\image_30.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_07_ace\image_01.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_07_ace\image_02.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_07_ace\image_03.png
C:/Users/Scri

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_20.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_21.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_22.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_23.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_24.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_25.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_26.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image_27.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_33_18_spades\image

...

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_18.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_19.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_20.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_21.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_22.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_23.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_24.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_25.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_17_six\image_26.png
C:/Users/Scrip0/Des

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_29_ten\image_27.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_29_ten\image_28.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_29_ten\image_29.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_01.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_02.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_03.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_04.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_05.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_43_34_queen\image_06.png
C:/User

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_18.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_19.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_20.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_21.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_22.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_23.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_24.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_25.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_30_four\image_26.png
C:/Users/S

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_17.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_18.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_19.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_20.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_21.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_22.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_23.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_24.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_41_eight\image_25.png
C

C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_04.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_05.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_06.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_07.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_08.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_09.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_10.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_11.png
C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/True\26_07_21_44_56_king\image_12.png
C:/Users/S

In [9]:
imgs, labels = shuffle(imgs, labels)

In [10]:
imgs = np.array(imgs) / 256.0
labels = np.array(labels)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, train_size=0.8, random_state=1) 

In [12]:
print(x_train.shape)
print(x_test.shape)

(1818, 128, 128, 3)
(455, 128, 128, 3)


In [13]:
model = keras.Sequential()

model.add(keras.Input(shape=(width, height, 3))) #3 if RGB, 1 if GrayScale

model.add(tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(16, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation="relu"))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                        

In [15]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss="MeanSquaredError")

In [16]:
early_stop = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=20, epochs=100, callbacks=[early_stop])

Epoch 1/100
91/91 [==============================] - 15s 77ms/step - loss: 0.1011 - val_loss: 0.1213
Epoch 2/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0947 - val_loss: 0.0935
Epoch 3/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0944 - val_loss: 0.1075
Epoch 4/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0938 - val_loss: 0.0850
Epoch 5/100
91/91 [==============================] - 5s 54ms/step - loss: 0.0942 - val_loss: 0.0864
Epoch 6/100
91/91 [==============================] - 5s 54ms/step - loss: 0.0921 - val_loss: 0.0852
Epoch 7/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0939 - val_loss: 0.0863
Epoch 8/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0926 - val_loss: 0.0843
Epoch 9/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0912 - val_loss: 0.0871
Epoch 10/100
91/91 [==============================] - 5s 55ms/step - loss: 0.0923 - val_loss: 0.084

In [17]:
model.save('Face_recognition.h5')

In [18]:
result = model.predict(x_test).flatten()
print(result)

15/15 [==============================] - 1s 31ms/step
[0.91756034 0.9510024  0.95810384 0.9484825  0.9051913  0.91926634
 0.9491066  0.9049472  0.9038973  0.95090014 0.7261535  0.72014505
 0.72385967 0.9490979  0.88426226 0.72112244 0.9005376  0.91541415
 0.9044182  0.8964187  0.8969003  0.9039195  0.86900103 0.936637
 0.9483648  0.9022188  0.94884866 0.9062835  0.8980204  0.8981187
 0.92474866 0.89132273 0.8816738  0.8662707  0.94638765 0.86958987
 0.9515122  0.8930205  0.7228089  0.9381544  0.92892325 0.91375023
 0.90352327 0.91198045 0.90251493 0.9557318  0.9042468  0.94302976
 0.93249124 0.896687   0.9250601  0.9128425  0.8709832  0.9203005
 0.9012934  0.90318036 0.94456106 0.93549716 0.9400528  0.88205624
 0.87775856 0.77006686 0.9035839  0.94376254 0.9150723  0.94678724
 0.89714444 0.9155927  0.91522944 0.8726378  0.9169372  0.9624757
 0.91963553 0.9051789  0.924562   0.72892976 0.913072   0.9418877
 0.89132404 0.8459389  0.909476   0.916607   0.91416466 0.9393456
 0.9125493  0.9